In [ ]:
%pip install pandas sqlalchemy

In [6]:
import pandas as pd

## Fast Finalization using SQL: Out-of-Core Data Querying



### Create the Database

Run the following code to create an SQLite3 file called 'gapminder.db', which contains the gapminder data.

This is the file we'll be pulling data from!

In [91]:
import bokeh
# bokeh.sampledata.download()
from bokeh.sampledata import gapminder
for name in ['population', 'fertility', 'life_expectancy', 'regions']:
    df = getattr(gapminder, name)
    if 'Group' in df:
        df = df.rename(columns={'Group': 'Continent'})
    df.to_sql(name, 'sqlite:///gapminder.db', if_exists='replace')


## Queries in SQL using Pandas

Reading in data from an sql database in Pandas is done using the SQLAlchemy library.  Instead of a filename, you'll need to give a [SQLAlchemy connection string](https://docs.sqlalchemy.org/en/14/core/engines.html#database-urls) to the read function:

| Function | Purpose | Example |
| :----    | :-----  | :------ |
| `pd.read_sql_table(table_name, conn_string)` | Reads a whole table in a database | `pd.read_sql_table('people', 'sqlite:///census.db'` |
| `pd.read_sql_query(sql, conn_string)` | Send an sql command to a database | `pd.read_sql_query('SELECT * FROM people', 'sqlite:///census.db'` |


### Exploring a Database File

Every type of SQL Datbase has a slightly different "dialect" of SQL (e.g. SQLite3, PostGreSQL, MySQL), but most SQL is similar.
The biggest difference in the dialect comes up when asking meta questions about the database, two of which are very important 
when just getting started:

   1. What Databases (e.g. "Schemas") can I get from this connection?
   2. What Tables are in this Schema?
   3. What Columns are in a table?
   
Each of this questions can be asked via a SQL query, and the answer is always a table of its own!

SQLite3 only can contain a single database, so we'll look at the last two questions:

| Question | Function | 
| :----    | :-----   |
| What Tables are there? | `SELECT * FROM sqlite_master` |
| What Columns are in this table? | `PRAGMA table_info('MyTable')`

**Exercises**

Use the `pd.read_sql_query()` function to answer the following questions about the organization of the "gapminder.db" SQLite3 database.

What Tables are in the database?

What columns are in the population table?

What columns are in the fertility table?

What columns are in the regions table?

### Reading a Full Table: `pd.read_sql_table(table_name, conn)`

If you know what table you want to read in, all you need is the table name and you're off!



**Exercises**

Use `pd.read_sql_table` to read each table from the database:

The fertility table:

The population table

The regions table

The life expectancy table

### Querying a Database: `pd.read_sql_query()`

If you have a more-detailed request you want to make, you can send an SQL query to the database
and get only the data you really want to work with.  Most queries in sql start with **SELECT**;
What kinds of queries can you make?  A lot! Here's a cheat sheet for basic queries:

[<img src="https://ugoproto.github.io/ugodoc/img/sql_nosql/SQL-Cheatsheet-2.png" width=620></img>](https://ugoproto.github.io/ugodoc/img/sql_nosql/SQL-Cheatsheet-2.png)


**Tip:**  These queries can get quite long; using python docstrings can help make it easer to read. for example:

```python
query = """
SELECT Name, Age FROM People
WHERE Age > 5
LIMIT 10
"""
pd.read_sql_query(query, 'sqlite:///data.db')
```

**Exercises**

Write SQL Queries that get only the data asked for below:

Example: Get all the fertility data.

In [199]:
query = """
SELECT * FROM fertility
"""
pd.read_sql_query(query, 'sqlite:///gapminder.db').head()

,Country,1964,1965,1966,1967,1968,1969,1970,1971,1972,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Afghanistan,7.671,7.671,7.671,7.671,7.671,7.671,7.671,7.671,7.671,...,7.136,6.930,6.702,6.456,6.196,5.928,5.659,5.395,5.141,4.900
1,Albania,5.711,5.594,5.483,5.376,5.268,5.160,5.050,4.933,4.809,...,2.004,1.919,1.849,1.796,1.761,1.744,1.741,1.748,1.760,1.771
2,Algeria,7.653,7.655,7.657,7.658,7.657,7.652,7.641,7.622,7.591,...,2.448,2.507,2.580,2.656,2.725,2.781,2.817,2.829,2.820,2.795
3,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Get all the popualtion data.

Get all the regions data

Print the 1980 fertility data.

Print the 2010 life expectancy data

Print the 1990 and 2010 and population data

Print Italy's Population data.

Print Germany's Population data.

Print Mexico and Cuba's 1990 fertility data

Print the first 3 rows of the life expectancy table

Print the first 3 rows of 1970's population data

Print the first 7 rows of 1990's fertility data

Show the Countries that had fertility rates above 6.5 people in 2004

Show the countries that had fertility rates below 1.4 in 2010.

Show the countries that had populations above 100 million in 1980. What were their populations in 2006?

Show the countries that are listed as being in "Europe & Central Asia"

### Joining Data

What if you want to compare and combine data from multiple tables?  The "Join" command is used the same way as Pandas' Merge function: comparing two columns and matching on them. 

Example: Get the fertility and population rates for all countries in 1980:

In [223]:
query = """
SELECT 
  p.Country, 
  p."1989" pop1989, 
  f."1989" fert1989 
FROM population p
INNER JOIN fertility f ON p.Country = f.Country
ORDER BY f."2010"
LIMIT 5
"""
pd.read_sql_query(query, 'sqlite:///gapminder.db')

,Country,pop1989,fert1989
0,American Samoa,45595.0,None
1,Andorra,51241.0,None
2,Anguilla,7964.0,None
3,Bermuda,59416.0,None
4,British Virgin Islands,15926.0,None


Get the life expectancy and fertility rates for all countries in 1990:

Get the life expectancy and fertility rates for all France in 1988:

Get the life expectancy in 2005 for all countries with fertility rates in 2005 less than 1.6.
Which countries are included?

Get the population of all countries with life expectancies of greater than 80 years in 2000.

Get the fertility rates of all countries in Europe

Get the population of all countries in Sub-saharan Africa

### Group By: Doing Statistics through SQL

https://www.sqlitetutorial.net/sqlite-group-by/

Aggregation functions: `AVG()`, `SUM()`, `MIN()`, `MAX()`

In [182]:
query = """
SELECT regions.Continent, regions.Country, AVG(population."1980") meanPop1980 FROM population
INNER JOIN regions ON population.Country = regions.Country
GROUP BY Continent
ORDER BY meanPop1980
"""
pd.read_sql_query(query, 'sqlite:///gapminder.db')

,Continent,Country,meanPop1980
0,Sub-Saharan Africa,Angola,7.636512e+06
1,Middle East & North Africa,Algeria,8.896094e+06
2,America,Anguilla,1.233459e+07
3,Europe & Central Asia,Albania,1.254172e+07
4,East Asia & Pacific,American Samoa,3.629319e+07
5,South Asia,Afghanistan,1.132596e+08


Which continents had the highest average population in 1990?

Which continents had the 2 lowest fertility rates in 2000?